In [4]:
with open("dhl_test.html", "r") as f:
    html = f.read()

In [9]:
import re
from bs4 import BeautifulSoup

def find_item(html):
    soup = BeautifulSoup(html, "html.parser")

    # Step 1: Find the <span> that contains the text "ARTIKEL"
    artikel_span = soup.find("span", class_="rio_15_grey", string=re.compile(r"\bARTIKEL\b"))
    if not artikel_span:
        return "Unknown item"

    # Step 2: Get the parent <tr> that holds this <span>
    artikel_tr = artikel_span.find_parent("tr")
    if not artikel_tr:
        return "Unknown item"

    # Step 3: Get the next <tr> sibling
    next_tr = artikel_tr.find_next_sibling("tr")
    if not next_tr:
        return "Unknown item"

    # OPTIONAL: If you only want a specific <span> inside the next <tr>
    # that might hold the item name, e.g. <span class="rio_15_heavy_black"> Reorda&reg; Metallband...</span>:
    item_span = next_tr.find("span", class_="rio_15_heavy_black")
    if item_span:
        # decode_contents(formatter="html") preserves &reg; instead of converting it to ®
        item_text = item_span.decode_contents(formatter="html").strip()
        return item_text

    return "Unknown item"


In [10]:
find_item(html)

'Reorda&reg; Metallband...'